# Import

In [ ]:
!pip install pandas

In [ ]:
subprocess.check_call([sys.executable, "-m", "pip", "install", "iteration_utilities"])

In [ ]:
import pandas as pd
import subprocess
from iteration_utilities import duplicates

On appelle le générateur de préférence 

In [ ]:
subprocess.call("generator.py", shell=True)

# Variables

Entrée : 

    Deux ensembles d'élèves et d'établissement avec leur préférence;

    Une famille L de relations de préférences ;

In [ ]:
prefEtudiant = pd.read_csv("fichierEleves.csv", header=None, sep = ",").values.tolist() 

prefEtablissement = pd.read_csv("fichierEtablissements.csv", header=None, sep = ",").values.tolist()

# Algorithme


Etape 0 : chaque élève soumet une liste ordonnée de vœux  
Etape 1 : on ne considère que les vœux de rang 1  
 * chaque école considère les élèves qui l’ont classée en 1er vœu  
 * chaque école accepte temporairement les mieux classés dans la limite des places disponibles et rejette les autres.  

. . .  

Etape k : les élèves rejetés à l’étape précédente candidatent
sur leur vœu suivant  
 * chaque école considère conjointement les élèves précédemment
admis et les élèves lui faisant une offre à cette étape  
 * les mieux classés sont temporairement acceptés et les autres
sont rejetés  
L’algorithme se termine au bout d’un nombre fini d’itérations
lorsque plus aucun élève n’est rejeté.

In [ ]:
def attribution(n, prefEtudiant, prefEtablissement):
    print("Tous les étudiants sont libres au début : ")
    etudiantLibre = list(range(n))
    print("Liste des étudiants libres : " ,etudiantLibre)
    # Aucun étudiant n'a d'affectation
    affectationEtudiant = [None] * n     
    print("affectation étudiant " ,affectationEtudiant)                 
    # Aucun établissement n'a d'étudiant
    affectationEtablissement = [None] * n  
    print("affectation établissement " ,affectationEtablissement)                    
    # Aucun étudiant n'a fait de proposition on est donc sur le choix 1 (0) de chaque étudiant
    suivant = [0] * n  
    print("suivant : " ,suivant)                     
    etape = 0
    # Tant que tous les étudiants n'ont pas d'affectation:
    while etudiantLibre:
        etape = etape +1
        print("  ")
        print("-------------------------------------------------------------------------------------------")
        print("  ")
        print("Etape : ",etape)
        # On prend le premier des étudiants lilbres
        etudiant = etudiantLibre[0] 
        print("On prend l'etudiant : ",etudiant)                     
        # On prend ses préférences
        prefEtudiantLibre = prefEtudiant[etudiant]  
        print("Préférences de l'étudiant ",etudiant," : ", prefEtudiantLibre)
        # On prend le choix n de l'étudiant
        etablissement = prefEtudiantLibre[suivant[etudiant]] 
        print("Le choix numéro : ",suivant[etudiant]," de l'étudiant :  ",etudiant ," est  : ", etablissement)
        # On prend le classement de l'élève
        prefEtablissementEleve = prefEtablissement[etablissement]
        print("Classement de l'élève : ", prefEtablissementEleve)
        # On prend l'étudiant actuellement affecté à cet établissement
        etudiantActuel = affectationEtablissement[etablissement]
       
        
        if etudiantActuel == None:
          #Si il n'y a pas d'étudiant déjà affecté à cet établissement la demande est accepté
          affectationEtablissement[etablissement] = etudiant
          affectationEtudiant[etudiant] = etablissement
          print("Aucun étudiant n'est déjà affecté a cet établissement.")
          print(etudiant," affecté à l'établissement : ", etablissement)
          print("Affectation des établissements : ",affectationEtablissement)
          print("Affectation des étudiants : ",affectationEtudiant)
          #Son choix suivant passe à la femme suivante dans sa liste
          suivant[etudiant] = suivant[etudiant] + 1
          #On l'enlève de la liste des étudiants libres
          etudiantLibre.pop(0)
        else:
          #Si il y a déjà un étudiant affecté à cet établissement
          print("Préférences de l'établissement : ", prefEtablissementEleve)
          print("L'étudiant : ", etudiantActuel," est déja affecté à l'établissement ", etablissement)

          # On regarde les préférences de l'établissement pour les 2 élèves
          indexEtudiantActuel = prefEtablissementEleve.index(etudiantActuel)
          
          indexEtudiant2 = prefEtablissementEleve.index(etudiant)
          
          if indexEtudiantActuel > indexEtudiant2:
             #L'établissement préfere le nouvel étudiant
             print(etudiant," est préféré à ", etudiantActuel)
             affectationEtablissement[etablissement] = etudiant
             print(etudiant," affecté à l'établissement : ", etablissement)
             affectationEtudiant[etudiant] = etablissement
             suivant[etudiant] = suivant[etudiant] + 1
             #On l'enlève de la liste des étudiants libres
             etudiantLibre.pop(0)
             #L'autre étudiant n'a plus d'établissement
             etudiantLibre.insert(0,etudiantActuel)
          else:
            print(etudiantActuel," est préféré à ", etudiant)
            print("Son choix ",suivant[etudiant]," est refusé")
            suivant[etudiant] = suivant[etudiant] + 1
             
           
    
  
    return affectationEtudiant

Sortie :

    Liste des affectations par couple

In [ ]:
res = attribution(len(prefEtudiant), prefEtudiant, prefEtablissement)
print("  ")
print("----------------------------------RESULTATS-----------------------------------------")
print("  ")
print(res)



On teste s'il n'y a pas de doublons

In [ ]:
print(list(duplicates(res)))
# Si la fonction renvoie [] il n'y a pas de doublons